# <center>『 밑바닥부터 시작하는 딥러닝 』</center>

<img src="http://image.kyobobook.co.kr/images/book/large/636/l9788968484636.jpg">

---

# 5장. 오차역전법

---

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#계산-그래프" data-toc-modified-id="계산-그래프-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>계산 그래프</a></span><ul class="toc-item"><li><span><a href="#계산-그래프로-풀다" data-toc-modified-id="계산-그래프로-풀다-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>계산 그래프로 풀다</a></span></li><li><span><a href="#국소적-계산" data-toc-modified-id="국소적-계산-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>국소적 계산</a></span></li><li><span><a href="#왜-계산-그래프로-푸는가?" data-toc-modified-id="왜-계산-그래프로-푸는가?-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>왜 계산 그래프로 푸는가?</a></span></li></ul></li><li><span><a href="#연쇄법칙" data-toc-modified-id="연쇄법칙-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>연쇄법칙</a></span><ul class="toc-item"><li><span><a href="#계산-그래프의-역전파" data-toc-modified-id="계산-그래프의-역전파-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>계산 그래프의 역전파</a></span></li><li><span><a href="#연쇄법칙이란?" data-toc-modified-id="연쇄법칙이란?-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>연쇄법칙이란?</a></span></li><li><span><a href="#연쇄법칙과-계산-그래프" data-toc-modified-id="연쇄법칙과-계산-그래프-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>연쇄법칙과 계산 그래프</a></span></li></ul></li><li><span><a href="#역전파" data-toc-modified-id="역전파-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>역전파</a></span><ul class="toc-item"><li><span><a href="#덧셈-노드의-역전파" data-toc-modified-id="덧셈-노드의-역전파-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>덧셈 노드의 역전파</a></span></li><li><span><a href="#곱셈-노드의-역전파" data-toc-modified-id="곱셈-노드의-역전파-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>곱셈 노드의 역전파</a></span></li><li><span><a href="#사과-쇼핑의-예" data-toc-modified-id="사과-쇼핑의-예-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>사과 쇼핑의 예</a></span></li></ul></li><li><span><a href="#단순한-계층-구현하기" data-toc-modified-id="단순한-계층-구현하기-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>단순한 계층 구현하기</a></span><ul class="toc-item"><li><span><a href="#곱셈-계층" data-toc-modified-id="곱셈-계층-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>곱셈 계층</a></span></li><li><span><a href="#덧셈-계층" data-toc-modified-id="덧셈-계층-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>덧셈 계층</a></span></li></ul></li><li><span><a href="#활성화-함수-계층-구현하기" data-toc-modified-id="활성화-함수-계층-구현하기-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>활성화 함수 계층 구현하기</a></span><ul class="toc-item"><li><span><a href="#ReLU-계층" data-toc-modified-id="ReLU-계층-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>ReLU 계층</a></span></li><li><span><a href="#시그모이드-계층" data-toc-modified-id="시그모이드-계층-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>시그모이드 계층</a></span></li></ul></li><li><span><a href="#Affine/Softmax-계층-구현하기" data-toc-modified-id="Affine/Softmax-계층-구현하기-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Affine/Softmax 계층 구현하기</a></span><ul class="toc-item"><li><span><a href="#Affine-계층" data-toc-modified-id="Affine-계층-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Affine 계층</a></span></li><li><span><a href="#배치용-Affine-계층" data-toc-modified-id="배치용-Affine-계층-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>배치용 Affine 계층</a></span></li><li><span><a href="#Softmax-with-Loss-계층" data-toc-modified-id="Softmax-with-Loss-계층-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>Softmax-with-Loss 계층</a></span></li></ul></li><li><span><a href="#오차역전파법-구현하기" data-toc-modified-id="오차역전파법-구현하기-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>오차역전파법 구현하기</a></span><ul class="toc-item"><li><span><a href="#신경망-학습의-전체-그림" data-toc-modified-id="신경망-학습의-전체-그림-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>신경망 학습의 전체 그림</a></span></li><li><span><a href="#오차-역전파법을-적용한-신경망-구현하기" data-toc-modified-id="오차-역전파법을-적용한-신경망-구현하기-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>오차 역전파법을 적용한 신경망 구현하기</a></span></li><li><span><a href="#오차역전파법으로-구한-기울기-검증하기" data-toc-modified-id="오차역전파법으로-구한-기울기-검증하기-7.3"><span class="toc-item-num">7.3&nbsp;&nbsp;</span>오차역전파법으로 구한 기울기 검증하기</a></span></li><li><span><a href="#오차역전파법을-사용한-학습-구현하기" data-toc-modified-id="오차역전파법을-사용한-학습-구현하기-7.4"><span class="toc-item-num">7.4&nbsp;&nbsp;</span>오차역전파법을 사용한 학습 구현하기</a></span></li></ul></li></ul></div>

--- 

4장에서는 신경망 학습에 대해서 공부했습니다. 그때 신경망의 가중치 매개변수의 기울기(가중치 매개변수에 대한 손실함수의 기울기)는 **수치 미분**을 사용해 구했습니다. 수치 미분은 단순하고 구현하기도 쉽지만 **계산 시간이 오래 걸린다**는 게 단점입니다.  

이번 장에서는 **가중치 매개변수의 기울기를 효율적으로 계산**하는 **오차역전파법(backpropagation)**을 배워보겠습니다.

**< 오차역전파법을 제대로 이해하는 방법 >**  
- **수식을 통한 것**  
일반적인 방법  
특히 기계학습을 다루는 책 대부분은 수식을 중심으로 이야기를 전개함  


- **계산 그래프를 통한 것**   
시각적으로 이해할 수 있음  
CS231n 강의를 참고함

## 계산 그래프

**계산 그래프(conputational graph)**는 계산 과정을 그래프로 나타낸 것입니다. 여기에서의 **그래프**는 우리가 잘 아는 **그래프 자료구조**로, 복수의 **노드(node)**와, **에지(edge)**로 표현됩니다.

### 계산 그래프로 풀다

- *문제 1: A군은 슈퍼에서 1개에 100원인 사과 2개를 샀습니다. 이때 지불 금액을 구하세요. 단, 소비세가 10% 부과됩니다.*
<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FdtNccc%2FbtqzWKCrGMN%2FWIqafEuO6i0TsHGuG5RfPk%2Fimg.png" width=70%>
위의 그림에서는 'x2'와 'x1.1'을 각각 하나의 연산으로 취급해 원 안에 표기했지만, 곱셈 기호(x)와 같은 **연산 기호만 연산으로 생각**할 수 있으며 그때의 계산 그래프는 다음과 같이 표기할 수 있다.  
<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbCGRMa%2FbtqzVDDKuSO%2FweFgDFWnB3NV9NFvvGKUx1%2Fimg.png" width=70%>

- *A군은 슈퍼에서 사과를 2개, 귤을 3개 샀습니다. 사과는 1개에 100원, 귤은 1개에 150원 입니다. 소비세가 10%일 때 지불 금액을 구하세요.*  
<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbMK9yY%2FbtqzVofTsov%2FWzWX1oYmTKnh3WQvKxfBr0%2Fimg.png" width=70%>

지금까지 살펴본 것처럼 계산 그래프를 이용한 문제풀이는 다음 흐름으로 진행합니다.  

1. 계산 그래프를 구성한다.  
2. 그래프에서 계산을 왼쪽에서 오른쪽으로 진행한다.  

여기서 2번째 **계산을 왼쪽에서 오른쪽으로 진행**하는 단계를 **순전파(forward propagation)**라고 합니다. 순전파는 계산 그래프의 출발점부터 종착점으로의 전파입니다. 순전파의 반대(오른쪽에서 왼쪽)의 전파를 **역전파(backward propagation)**라고 합니다. 역전파는 이후에 **미분을 계산할 때 중요한 역할**을 합니다.

### 국소적 계산

계산 그래프의 특징은 **국소적 계산**을 전파함으로써 최종 결과를 얻는다는 점에 있습니다. 국소적이란 자신과 직접 관계된 작은 범위라는 뜻입니다. 

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FSNwQI%2FbtqzVoNF34I%2FF92KQNgYBVmjOKgkWKkZnk%2Fimg.png" width=70%>

4000 + 200 이라는 계산은 4000과 200 각각이 어떻게 계산되었느냐와는 상관없이, 단지 두 숫자를 더하면 된다는 뜻입니다. **각 노드는 자신과 관련한  계산 외에는 아무것도 신경 쓸 게 없습니다. 이처럼 국소적인 계산은 단순하지만, 그 결과를 전달함으로써 전체를 구성하는 복잡한 계산을 해낼 수 있습니다.

### 왜 계산 그래프로 푸는가?

**< 계산 그래프의 이점 >**  

1. 국소적 계산
2. 중간 계산 결과를 모두 보관할 수 있음
3. **(가장 큰 이유) 역전파를 통해 '미분'을 효율적으로 계산할 수 있음**

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=http%3A%2F%2Fcfile1.uf.tistory.com%2Fimage%2F997E914D5B98F628261EF4">

빨간색의 역전파는 국소적 미분을 전달하고 그 미분 값은 화살표의 아래에 적습니다.
- 1.1 = 1 x (220 / 200)
- 2.2 = 1.1 x (200 / 100)

## 연쇄법칙

역전파는 '국소적인 미분'을 오른쪽에서 왼쪽으로 전달합니다. 그 원리는 **연쇄법칙(chain rule)**에 따른 것입니다.

### 계산 그래프의 역전파

**y = f(x)** 라는 계산의 역전파를 그려봤습니다.

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=http%3A%2F%2Fcfile8.uf.tistory.com%2Fimage%2F999FD3425B98F63F1A9DDF">

### 연쇄법칙이란?

<img src="https://blog.kakaocdn.net/dn/ClmZy/btqUWsbh7ES/2YIq09rKMIa3PyHwGeGLkk/img.png">

### 연쇄법칙과 계산 그래프

맨 왼쪽의 역전파 계산은 연쇄법칙에 따라 **x에 대한 z의 미분**이 됩니다. 즉, 역전파가 하는 일은 연쇄법칙의 원리와 같다는 겁니다.

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=http%3A%2F%2Fcfile27.uf.tistory.com%2Fimage%2F997387465B98F65D133C64">

## 역전파

앞 절에서는 계산 그래프의 역전파가 연쇄법칙에 따라 진행되는 모습을 설명했습니다. 이번 절에서는 '+'와 'x' 등의 연산을 예로 들어 **역전파의 구조**를 설명합니다.

### 덧셈 노드의 역전파

덧셈 노드의 역전파는 **입력 신호를 그대로 다음 노드로 전달**합니다.

<img src="https://blog.kakaocdn.net/dn/bEHO2G/btqUYfo9jTQ/1P5EdrS608L0voayN8kUX0/img.png">

### 곱셈 노드의 역전파

곱셈 노드 역전파는 상류의 값에 순전파 때의 입력 신호들을 **서로 바꾼 값**을 곱해서 하류로 보냅니다.  

곱셈의 역전파는 순방향 입력 신호의 값이 필요합니다. 그래서 곱셈 노드를 구현할 때는 순전파의 입력 신호를 변수에 저장해둡니다.

<img src="https://blog.kakaocdn.net/dn/oIkN9/btqUTnIihIj/6KDd13Ya8YiqhLmpQSoTA1/img.png">

### 사과 쇼핑의 예

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FUd7EE%2FbtqAIDJ0hnW%2FMWJWr1eSWkFK9apWgqN0fk%2Fimg.png" width=70%>

## 단순한 계층 구현하기

### 곱셈 계층

In [1]:
class MulLayer:
    
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        
        out = x * y
        
        return out
    
    def backward(self, dout):
        dx = dout * self.y # x와 y를 바꾼다.
        dy = dout * self.x
        
        return dx, dy

In [2]:
apple = 100
apple_num = 2
tax = 1.1

In [3]:
# 계층들
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

In [4]:
# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

220.00000000000003


In [5]:
# 역전파

dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print("price:", int(price))
print("dApple:", dapple)
print("dApple_num:", int(dapple_num))
print("dTax:", dtax)

price: 220
dApple: 2.2
dApple_num: 110
dTax: 200


### 덧셈 계층

In [6]:
class AddLayer:
    def __init__(self):
        pass

    def forward(self, x, y):
        out = x + y

        return out

    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1

        return dx, dy

In [7]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)  # (1)
orange_price = mul_orange_layer.forward(orange, orange_num)  # (2)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)  # (3)
price = mul_tax_layer.forward(all_price, tax)  # (4)

# backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)  # (4)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)  # (3)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)  # (2)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)  # (1)

print("price:", int(price))
print("dApple:", dapple)
print("dApple_num:", int(dapple_num))
print("dOrange:", dorange)
print("dOrange_num:", int(dorange_num))
print("dTax:", dtax)

price: 715
dApple: 2.2
dApple_num: 110
dOrange: 3.3000000000000003
dOrange_num: 165
dTax: 650


## 활성화 함수 계층 구현하기

계산 그래프를 신경마엥 적용합니다. 여기에서는 신경망을 구성하는 층 각각을 클래스 하나로 구현합니다.  
우선은 **활성화 함수**인 **ReLU**와 **Sigmoid** 계층을 구현하겠습니다.

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FvnQPJ%2FbtqEd7HdyUh%2FETBAZp5B17K8KiwAleT3HK%2Fimg.png" width=70%>

활성화 함수 관련 추가 링크: https://deepinsight.tistory.com/113

### ReLU 계층

활성화 함수로 사용되는 **ReLU**의 수식은 다음과 같습니다.  
<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbDxdMs%2FbtqAJhUyVEu%2F6xCtrkc6NUNH7J6cHH98m0%2Fimg.png" width=30%>

순전파 때의 입력인 x가 0보다 크면 역전파는 상류의 값을 그대로 하류로 흘립니다. 반면, 순전파 때 x가 0 이하면 역전파 때는 하류로 신호를 보내지 않습니다.(0을 보냅니다.)

In [8]:
class Relu:
    
    def __init__(self):
        self.mask = None

    def forward(self, x): 
        self.mask = (x <= 0) 
        out = x.copy() 
        out[self.mask] = 0 
        
        return out 
    
    def backward(self, dout): 
        dout[self.mask] = 0 
        dx = dout 
        
        return dx

In [9]:
import numpy as np

In [10]:
x = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)

[[ 1.  -0.5]
 [-2.   3. ]]


In [11]:
mask = (x <= 0)
print(mask)

[[False  True]
 [ True False]]


### 시그모이드 계층

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fde9qfI%2FbtqEeqfuVyY%2FGQl00vB6WW0K9MLYBZ2Zxk%2Fimg.png">

**시그모이드 함수**의 수식은 다음과 같습니다.  
<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbuXy9x%2FbtqALQgKeCz%2FREaaWJwaxHwamvvo9rBND0%2Fimg.png" width=40%>

In [12]:
class Sigmoid: 
    
    def __init__(self): 
        self.out = None 
    
    def forward(self, x): 
        out = sigmoid(x) 
        self.out = out 
        
        return out 
    
    def backward(self, dout): 
        dx = dout * (1.0 - self.out) * self.out 
        
        return dx

시그모이드 함수는 다음과 같은 문제점이 있습니다.  
1. Saturated
2. Npt zero-centered
<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2F2WSMx%2FbtqEfqzjiXL%2F9XjdUjuG5yM22GYsCFzFb0%2Fimg.png">

## Affine/Softmax 계층 구현하기

### Affine 계층

**Affine 계층**은 순전파에서 수행하는 행렬의 내적을 기하학에서 부르는 말입니다. Input 값과 weight 값들을 행렬 곱하여 거기에 편향을 추가하여 출력값 y를 최종적으로 반환하는 것입니다.

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbBHb0k%2FbtqALvjUUVe%2FP4srUaWP5vSp7gAXCnNfh0%2Fimg.png" width=70%>

### 배치용 Affine 계층

앞서 설명한 Affine 계층은 입력 데이터로 X 하나만을 고려한 것이었습니다. **배치용 Affine 계층**이란 데이터 N개를 묶어 순전파하는 경우, 즉 배치용 Affine 계층을 생각해보겠습니다.

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fbg7kNw%2FbtqAK4tisBq%2F8uKeKfZdqqU8KKaLCEXF4k%2Fimg.png" width=70%>

### Softmax-with-Loss 계층

출력층에서 사용하는 소프트맥스 함수에 관해 설명하겠습니다. 소프트맥스 함수는 입력 값을 정규화하여 출력합니다.  
입력받은 값을 0~1 사이의 출력이 되도록 정규화하여 **출력 값들의 총합이 항상 1이 되는 특성**을 가진 함수입니다. 소프트맥스는 보통 딥러닝에서 **출력 노드의 활성화 함수**로 많이 사용됩니다.

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FzGPE4%2FbtqALQH9zso%2FbpLtNaGnlk61XWurzui000%2Fimg.png">

추론할 때는 일반적으로 Softmax 계층을 사용하지 않습니다. Softmax 앞의 Affine 계층의 출력(점수, score)를 사용합니다. 반면, 신경망을 학습할 때는 Softmax 계층이 필요합니다.

소프트맥스 계층을 구현할 때에는 손실 함수인 **교차 엔트로피 오차**도 포함하기 때문에 **Softmax-with-Loss 계층**이라는 이름으로 구현합니다.

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FcTrT4B%2FbtqAI33FH4t%2FkG3MbBq7Bl1uLDrkMnWF61%2Fimg.png">

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbzX4Z3%2FbtqALt0HJ6j%2FyHOuB9073HnOwhgxlDExkK%2Fimg.png" width=70%>

여기에서는 3 클래스 분류를 가정하고 이전 계층에서 3개의 입력(점수)를 받습니다.  
소프트맥스 계층은 입력(a1, a2, a3)를 정규화하여 0~1 사이의 값인 (y1, y2, y3)를 출력합니다. 교차 엔트로피 계층은 softmax의 출력 (y1, y2, y3)와 정답 레이블(t1, t2, t3)를 받고, 이 데이터들로부터 손실 L을 출력합니다.

여기서 주목할 점은 소프트맥스 계층의 역전파는 (y1-t1, y2-t2, y3-t3)라는 말끔한 결과를 내놓고 있습니다. 이는 softmax 계층의 출력과 정답 레이블의 차분입니다. 신경망의 역전파에서는 이 차이인 오차가 앞 계층에 전해지는 것입니다.

In [13]:
class SoftmaxWithLoss: 
    
    def __init__(self): 
        self.loss = None # 손실 
        self.y = None # softmax의 출력 
        self.t = None # 정답 레이블(원-핫 벡터) 
        
    def forward(self, x, t): 
        self.t = t 
        self.y = softmax(x) 
        self.loss = corss_entropy_error(self.y, self.t) 
        return self.loss 
    
    def backward(self, dout=1): 
        batch_size = self.t.shape[0] 
        dx = (self.y - self.t) / batch_Size 
        
        return dx

## 오차역전파법 구현하기

지금부터는 이전에 구현한 계층을 조합해서 신경망을 구축해보겠습니다.

### 신경망 학습의 전체 그림

- **전제**  
신경망에는 적응 가능한 **가중치**와 **편향**이 있고, 이 가중치와 편향을 훈련 데이터에 적응하도록 조정하는 과정을 **학습**이라고 합니다. 신경망은 다음과 같이 4단계로 수행합니다.  


- **1단계: 미니배치**  
훈련 데이터 중 일부를 무작위로 가져옵니다. 이렇게 선별한 데이터를 **미니배치**라 하며, 그 미니배치의 **손실 함수 값을 줄이는 것이 목표**입니다.  


- **2단계: 기울기 산출**  
미니배치의 손실 함수 값을 줄이기 위해 **각 가중치 매개변수의 기울기**를 구합니다. 기울기는 손실함수의 값을 가장 작게 하는 방향을 제시합니다.    


- **3단계: 매개변수 갱신**  
가중치 매개변수를 기울기 방향으로 아주 조금 갱신합니다.  


- **4단계**  
1 ~ 3 단계르 반복합니다.

**오차역전파법**이 등장하는 단계는 2단계(기울기 산출) 입니다.

### 오차 역전파법을 적용한 신경망 구현하기

여기서는 2층 신경망을 TwoLayerNet 클래스로 구현합니다.

<img src="https://blog.kakaocdn.net/dn/cGjpzC/btqVh7Sb0o6/HCNwQHrBd80oMcVq1LIEM1/img.png">

In [14]:
class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
        self.params['b2'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
        
    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads

### 오차역전파법으로 구한 기울기 검증하기

In [15]:
# pass (p.184~185)

### 오차역전파법을 사용한 학습 구현하기

마지막으로 오차역전파법을 사용한 신경망 학습을 구현해보겠습니다. 지금까지와 다른 부분은 **기울기를 오차역전파법으로 구한다**는 점뿐입니다.

In [16]:
"""
import sys, os
sys.path.append(os.pardir)

import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet
"""

'\nimport sys, os\nsys.path.append(os.pardir)\n\nimport numpy as np\nfrom dataset.mnist import load_mnist\nfrom two_layer_net import TwoLayerNet\n'

In [17]:
"""
# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 계산
    #grad = network.numerical_gradient(x_batch, t_batch) # 수치 미분 방식
    grad = network.gradient(x_batch, t_batch) # 오차역전파법 방식(훨씬 빠르다)
    
    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)
        
"""

"\n# 데이터 읽기\n(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)\n\nnetwork = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)\n\niters_num = 10000\ntrain_size = x_train.shape[0]\nbatch_size = 100\nlearning_rate = 0.1\n\ntrain_loss_list = []\ntrain_acc_list = []\ntest_acc_list = []\n\niter_per_epoch = max(train_size / batch_size, 1)\n\nfor i in range(iters_num):\n    batch_mask = np.random.choice(train_size, batch_size)\n    x_batch = x_train[batch_mask]\n    t_batch = t_train[batch_mask]\n    \n    # 기울기 계산\n    #grad = network.numerical_gradient(x_batch, t_batch) # 수치 미분 방식\n    grad = network.gradient(x_batch, t_batch) # 오차역전파법 방식(훨씬 빠르다)\n    \n    # 갱신\n    for key in ('W1', 'b1', 'W2', 'b2'):\n        network.params[key] -= learning_rate * grad[key]\n    \n    loss = network.loss(x_batch, t_batch)\n    train_loss_list.append(loss)\n    \n    if i % iter_per_epoch == 0:\n        train_acc = network.accuracy(x_train, t_train)\n        te